In [1]:
# Dependencies
import json
import requests
import pandas as pd
from scipy import stats
from api_keys import api_key_eia,api_key_av
import matplotlib.pyplot as plt

## Prepare the montly change of crude oil price dataframe

In [2]:
# Read the original CSV file of crude oil WTI futures price (daily basis)
oil_daily = pd.read_csv("Resources/oil_futures_daily.csv").rename(columns = {"value" : "oil_spot_price"})
oil_daily.head(11)

,date,oil_spot_price
0,2023-09-25,89.68
1,2023-09-22,90.0
2,2023-09-21,89.56
3,2023-09-20,89.2
4,2023-09-19,91.16
5,2023-09-18,91.47
6,2023-09-15,90.83
7,2023-09-14,90.13
8,2023-09-13,88.59
9,2023-09-12,88.87


In [3]:
# Replace non-numeric values with NaN in the 'oil_spot_price' column
oil_daily['oil_spot_price'] = oil_daily['oil_spot_price'].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN values in the 'oil_spot_price' column
oil_daily.dropna(subset=['oil_spot_price'], inplace=True)

In [4]:
# Review the data frame
oil_daily.head(11)

,date,oil_spot_price
0,2023-09-25,89.68
1,2023-09-22,90.00
2,2023-09-21,89.56
3,2023-09-20,89.20
4,2023-09-19,91.16
5,2023-09-18,91.47
6,2023-09-15,90.83
7,2023-09-14,90.13
8,2023-09-13,88.59
9,2023-09-12,88.87


In [6]:
# Check the data type
oil_daily.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9506 entries, 9842 to 0
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            9506 non-null   object 
 1   oil_spot_price  9506 non-null   float64
 2   Diff            9505 non-null   float64
dtypes: float64(2), object(1)
memory usage: 297.1+ KB


In [7]:
# Convert the "date" column to datetime data type
oil_daily['date'] = pd.to_datetime(oil_daily['date'])

In [8]:
# Check the data type again
oil_daily.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9506 entries, 9842 to 0
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            9506 non-null   datetime64[ns]
 1   oil_spot_price  9506 non-null   float64       
 2   Diff            9505 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 297.1 KB


In [9]:
# Group by week and average the total weekly oil spot price
average_weekly = oil_daily.groupby(pd.Grouper(key='date', freq='W'))['oil_spot_price'].mean().reset_index()
average_weekly.head()

,date,oil_spot_price
0,1986-01-05,25.780
1,1986-01-12,25.986
2,1986-01-19,24.568
3,1986-01-26,20.314
4,1986-02-02,19.692


In [10]:
# Reset the index and rename the column name 
average_weekly = average_weekly.rename(columns={'oil_spot_price':'oil_spot_price_weekly'})
average_weekly.tail()

,date,oil_spot_price_weekly
1965,2023-09-03,82.5000
1966,2023-09-10,87.1675
1967,2023-09-17,89.1440
1968,2023-09-24,90.2780
1969,2023-10-01,89.6800


In [11]:
# Filter the results to include only dates from Jun 2010 to Sep 2023
f_average_weekly = average_weekly[(average_weekly['date'] >= '2010-06-06')&
                                  (average_weekly['date'] <= '2023-09-24')].reset_index(drop=True)
f_average_weekly.tail()

,date,oil_spot_price_weekly
690,2023-08-27,79.9720
691,2023-09-03,82.5000
692,2023-09-10,87.1675
693,2023-09-17,89.1440
694,2023-09-24,90.2780


In [13]:
# Create a new dataframe for training and testing machine model
oil_model_weekly_df = f_average_weekly.copy()
oil_model_weekly_df.head()

,date,oil_spot_price_weekly
0,2010-06-06,72.9075
1,2010-06-13,73.4360
2,2010-06-20,76.7000
3,2010-06-27,77.0600
4,2010-07-04,74.9580


## Collecting crude oil supply/demand data from EIA
- Note: Due to the limitations of API calls,the full data needs to be collected separately
---
### 1.Weekly Refiner Net Production

In [14]:
# Set up API query for weekly refiner net production (from 2010-06-04 t0 2014-06-06)
api_url = f"https://api.eia.gov/v2/petroleum/pnp/wprodr/data/?frequency=weekly&data[0]=value&start=2010-06-04&end=2014-06-06&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
refiner_data1 = r.json()

In [15]:
print(refiner_data1)

{'response': {'total': 4620, 'dateFormat': 'YYYY-MM-DD', 'frequency': 'weekly', 'data': [{'period': '2014-06-06', 'duoarea': 'NUS', 'area-name': 'U.S.', 'product': 'EPD0', 'product-name': 'Distillate Fuel Oil', 'process': 'YPY', 'process-name': 'Refinery Net Production', 'series': 'W_EPD0_YPY_NUS_MBBLD', 'series-description': 'U.S. Refiner Net Production of Distillate Fuel Oil (Thousand Barrels per Day)', 'value': 4824, 'units': 'MBBL/D'}, {'period': '2014-06-06', 'duoarea': 'NUS', 'area-name': 'U.S.', 'product': 'EPPK', 'product-name': 'Kerosene', 'process': 'YPY', 'process-name': 'Refinery Net Production', 'series': 'W_EPPK_YPY_NUS_MBBLD', 'series-description': 'U.S. Refiner Net Production of Kerosene (Thousand Barrels per Day)', 'value': -1, 'units': 'MBBL/D'}, {'period': '2014-06-06', 'duoarea': 'NUS', 'area-name': 'U.S.', 'product': 'EPM0RA', 'product-name': 'Reformulated Motor Gasoline with Alcohol', 'process': 'YPY', 'process-name': 'Refinery Net Production', 'series': 'W_EPM0RA

In [16]:
# Convert the data into DataFrame
refiner_df1 = pd.DataFrame(refiner_data1['response']['data'])
refiner_df1.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2014-06-06,NUS,U.S.,EPD0,Distillate Fuel Oil,YPY,Refinery Net Production,W_EPD0_YPY_NUS_MBBLD,U.S. Refiner Net Production of Distillate Fuel...,4824,MBBL/D
1,2014-06-06,NUS,U.S.,EPPK,Kerosene,YPY,Refinery Net Production,W_EPPK_YPY_NUS_MBBLD,U.S. Refiner Net Production of Kerosene (Thous...,-1,MBBL/D
2,2014-06-06,NUS,U.S.,EPM0RA,Reformulated Motor Gasoline with Alcohol,YPY,Refinery Net Production,W_EPM0RA_YPY_NUS_MBBLD,U.S. Refiner Net Production of Reformulated Mo...,58,MBBL/D
3,2014-06-06,NUS,U.S.,EPDXL0,"Distillate Fuel Oil, 0 to 15 ppm Sulfur",YPY,Refinery Net Production,W_EPDXL0_YPY_NUS_MBBLD,U.S. Refiner Net Production of Distillate Fuel...,4473,MBBL/D
4,2014-06-06,NUS,U.S.,EPM0CAG55,Finished Motor Gasoline Conventional >55,YPY,Refinery Net Production,W_EPM0CAG55_YPY_NUS_MBBLD,U.S. Refiner Net Production of Finished Conven...,0,MBBL/D


In [17]:
# Set up API query for weekly refiner net production (from 2014-06-07 to 2018-09-30)
api_url = f"https://api.eia.gov/v2/petroleum/pnp/wprodr/data/?frequency=weekly&data[0]=value&start=2014-06-07&end=2018-09-30&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
refiner_data2 = r.json()

In [18]:
# Convert the data into DataFrame
refiner_df2 = pd.DataFrame(refiner_data2['response']['data'])
refiner_df2.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2018-09-28,NUS,U.S.,EPM0CAL55,"Motor Gasoline, Finished, Conventional, Ed55 a...",YPY,Refinery Net Production,W_EPM0CAL55_YPY_NUS_MBBLD,"U.S. Refiner Net Production of Motor Gasoline,...",429,MBBL/D
1,2018-09-28,NUS,U.S.,EPJK,Kerosene-Type Jet Fuel,YPY,Refinery Net Production,W_EPJK_YPY_NUS_MBBLD,U.S. Refiner Net Production of Kerosene-Type J...,1744,MBBL/D
2,2018-09-28,NUS,U.S.,EPDM10,"Distillate Fuel Oil, Greater than 15 to 500 pp...",YPY,Refinery Net Production,W_EPDM10_YPY_NUS_MBBLD,U.S. Refiner Net Production of Distillate Fuel...,124,MBBL/D
3,2018-09-28,NUS,U.S.,EPM0R,Reformulated Motor Gasoline,YPY,Refinery Net Production,W_EPM0R_YPY_NUS_MBBLD,U.S. Refiner Net Production of Reformulated Mo...,47,MBBL/D
4,2018-09-28,NUS,U.S.,EPDXL0,"Distillate Fuel Oil, 0 to 15 ppm Sulfur",YPY,Refinery Net Production,W_EPDXL0_YPY_NUS_MBBLD,U.S. Refiner Net Production of Distillate Fuel...,4713,MBBL/D


In [19]:
# Set up API query for weekly refiner net production (from 2018-10-01 t0 2023-01-31)
api_url = f"https://api.eia.gov/v2/petroleum/pnp/wprodr/data/?frequency=weekly&data[0]=value&start=2018-10-01&end=2023-01-31&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
refiner_data3 = r.json()

In [20]:
# Convert the data into DataFrame
refiner_df3 = pd.DataFrame(refiner_data3['response']['data'])
refiner_df3.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2023-01-27,NUS,U.S.,EPM0RA,Reformulated Motor Gasoline with Alcohol,YPY,Refinery Net Production,W_EPM0RA_YPY_NUS_MBBLD,U.S. Refiner Net Production of Reformulated Mo...,40,MBBL/D
1,2023-01-27,NUS,U.S.,EPM0CAG55,Finished Motor Gasoline Conventional >55,YPY,Refinery Net Production,W_EPM0CAG55_YPY_NUS_MBBLD,U.S. Refiner Net Production of Finished Conven...,0,MBBL/D
2,2023-01-27,NUS,U.S.,EPDXL0,"Distillate Fuel Oil, 0 to 15 ppm Sulfur",YPY,Refinery Net Production,W_EPDXL0_YPY_NUS_MBBLD,U.S. Refiner Net Production of Distillate Fuel...,4464,MBBL/D
3,2023-01-27,NUS,U.S.,EPPR,Residual Fuel Oil,YPY,Refinery Net Production,W_EPPR_YPY_NUS_MBBLD,U.S. Refiner Net Production of Residual Fuel O...,267,MBBL/D
4,2023-01-27,NUS,U.S.,EPPK,Kerosene,YPY,Refinery Net Production,W_EPPK_YPY_NUS_MBBLD,U.S. Refiner Net Production of Kerosene (Thous...,11,MBBL/D


In [21]:
# Set up API query for weekly refiner net production (from 2023-02-01 t0 2023-09-22)
api_url = f"https://api.eia.gov/v2/petroleum/pnp/wprodr/data/?frequency=weekly&data[0]=value&start=2023-02-01&end=2023-09-22&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
refiner_data4 = r.json()

In [22]:
# Convert the data into DataFrame
refiner_df4 = pd.DataFrame(refiner_data4['response']['data'])
refiner_df4.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2023-09-22,NUS,U.S.,EPM0C,Conventional Motor Gasoline,YPY,Refinery Net Production,W_EPM0C_YPY_NUS_MBBLD,U.S. Refiner Net Production of Conventional Mo...,1293,MBBL/D
1,2023-09-22,NUS,U.S.,EPM0R,Reformulated Motor Gasoline,YPY,Refinery Net Production,W_EPM0R_YPY_NUS_MBBLD,U.S. Refiner Net Production of Reformulated Mo...,39,MBBL/D
2,2023-09-22,NUS,U.S.,EPJK,Kerosene-Type Jet Fuel,YPY,Refinery Net Production,W_EPJK_YPY_NUS_MBBLD,U.S. Refiner Net Production of Kerosene-Type J...,1772,MBBL/D
3,2023-09-22,R10,PADD 1,EPM0F,Finished Motor Gasoline,YPY,Refinery Net Production,W_EPM0F_YPY_R10_MBBLD,East Coast (PADD 1) Refiner Net Production of ...,54,MBBL/D
4,2023-09-22,R30,PADD 3,EPM0F,Finished Motor Gasoline,YPY,Refinery Net Production,W_EPM0F_YPY_R30_MBBLD,Gulf Coast (PADD 3) Refiner Net Production of ...,687,MBBL/D


In [23]:
# Combined all refiner dataframes vertically
refiner_combined = pd.concat([refiner_df1, refiner_df2, refiner_df3, refiner_df4], axis=0)
refiner_combined.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2014-06-06,NUS,U.S.,EPD0,Distillate Fuel Oil,YPY,Refinery Net Production,W_EPD0_YPY_NUS_MBBLD,U.S. Refiner Net Production of Distillate Fuel...,4824,MBBL/D
1,2014-06-06,NUS,U.S.,EPPK,Kerosene,YPY,Refinery Net Production,W_EPPK_YPY_NUS_MBBLD,U.S. Refiner Net Production of Kerosene (Thous...,-1,MBBL/D
2,2014-06-06,NUS,U.S.,EPM0RA,Reformulated Motor Gasoline with Alcohol,YPY,Refinery Net Production,W_EPM0RA_YPY_NUS_MBBLD,U.S. Refiner Net Production of Reformulated Mo...,58,MBBL/D
3,2014-06-06,NUS,U.S.,EPDXL0,"Distillate Fuel Oil, 0 to 15 ppm Sulfur",YPY,Refinery Net Production,W_EPDXL0_YPY_NUS_MBBLD,U.S. Refiner Net Production of Distillate Fuel...,4473,MBBL/D
4,2014-06-06,NUS,U.S.,EPM0CAG55,Finished Motor Gasoline Conventional >55,YPY,Refinery Net Production,W_EPM0CAG55_YPY_NUS_MBBLD,U.S. Refiner Net Production of Finished Conven...,0,MBBL/D


In [24]:
# Group by week and sum the total weekly refining volume
refiner_total = refiner_combined.groupby(by='period')['value'].sum()
refiner_total

period
2010-06-04    24800
2010-06-11    24507
2010-06-18    24736
2010-06-25    25214
2010-07-02    24559
              ...  
2023-08-25    17730
2023-09-01    17726
2023-09-08    17774
2023-09-15    18127
2023-09-22    17739
Name: value, Length: 695, dtype: int64

In [25]:
# Reset the index and rename the column name 
refiner_weekly_df = refiner_total.reset_index().rename(columns={'value':'weekly_total'})
refiner_weekly_df.tail()

,period,weekly_total
690,2023-08-25,17730
691,2023-09-01,17726
692,2023-09-08,17774
693,2023-09-15,18127
694,2023-09-22,17739


### 2. Weekly Blender Net Production

In [26]:
# Set up API query for weekly blender net production (from 2010-06-04 t0 2014-11-30)
api_url = f"https://api.eia.gov/v2/petroleum/pnp/wprodb/data/?frequency=weekly&data[0]=value&start=2010-06-04&end=2014-11-30&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
blender_data = r.json()

In [27]:
# Convert the data into DataFrame
blending_df1 = pd.DataFrame(blender_data['response']['data'])
blending_df1.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2014-11-28,NUS,U.S.,EPD0,Distillate Fuel Oil,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPD0_YPB_NUS_MBBLD,U.S. Blender Net Production of Distillate Fuel...,21,MBBL/D
1,2014-11-28,R20,PADD 2,EPM0F,Finished Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0F_YPB_R20_MBBLD,Midwest (PADD 2) Blender Net Production of Fin...,2166,MBBL/D
2,2014-11-28,NUS,U.S.,EPM0CO,Other Conventional Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0CO_YPB_NUS_MBBLD,U.S. Blender Net Production of Other Conventio...,-348,MBBL/D
3,2014-11-28,NUS,U.S.,EPD00H,"Distillate Fuel Oil, Greater Than 500 ppm Sulfur",YPB,Motor Gasoline Terminal Blenders Net Production,W_EPD00H_YPB_NUS_MBBLD,U.S. Blender Net Production of Distillate Fuel...,-21,MBBL/D
4,2014-11-28,NUS,U.S.,EPM0R,Reformulated Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0R_YPB_NUS_MBBLD,U.S. Blender Net Production of Reformulated Mo...,2989,MBBL/D


In [28]:
# Set up API query for weekly blender net production (from 2014-12-01 to 2019-05-31)
api_url = f"https://api.eia.gov/v2/petroleum/pnp/wprodb/data/?frequency=weekly&data[0]=value&start=2014-12-01&end=2019-05-31&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
blender_data2 = r.json()

In [29]:
# Convert the data into DataFrame
blending_df2 = pd.DataFrame(blender_data2['response']['data'])
blending_df2.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2019-05-31,NUS,U.S.,EPM0CAG55,Finished Motor Gasoline Conventional >55,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0CAG55_YPB_NUS_MBBLD,"U.S. Blender Net Production of Motor Gasoline,...",5,MBBL/D
1,2019-05-31,NUS,U.S.,EPDXL0,"Distillate Fuel Oil, 0 to 15 ppm Sulfur",YPB,Motor Gasoline Terminal Blenders Net Production,W_EPDXL0_YPB_NUS_MBBLD,U.S. Blender Net Production of Distillate Fuel...,36,MBBL/D
2,2019-05-31,NUS,U.S.,EPM0RO,Finished Motor Gasoline Reformulated Other,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0RO_YPB_NUS_MBBLD,U.S. Blender Net Production of Other Reformula...,0,MBBL/D
3,2019-05-31,NUS,U.S.,EPM0R,Reformulated Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0R_YPB_NUS_MBBLD,U.S. Blender Net Production of Reformulated Mo...,3168,MBBL/D
4,2019-05-31,R50,PADD 5,EPM0F,Finished Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0F_YPB_R50_MBBLD,West Coast (PADD 5) Blender Net Production of ...,1469,MBBL/D


In [30]:
# Set up API query for weekly blender net production (from 2019-06-01 to 2023-09-22)
api_url = f"https://api.eia.gov/v2/petroleum/pnp/wprodb/data/?frequency=weekly&data[0]=value&start=2019-06-01&end=2023-09-22&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
blender_data3 = r.json()

In [31]:
# Convert the data into DataFrame
blending_df3 = pd.DataFrame(blender_data3['response']['data'])
blending_df3.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2023-09-22,NUS,U.S.,EPDM10,"Distillate Fuel Oil, Greater than 15 to 500 pp...",YPB,Motor Gasoline Terminal Blenders Net Production,W_EPDM10_YPB_NUS_MBBLD,U.S. Blender Net Production of Distillate Fuel...,4,MBBL/D
1,2023-09-22,NUS,U.S.,EPM0C,Conventional Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0C_YPB_NUS_MBBLD,U.S. Blender Net Production of Conventional Mo...,5327,MBBL/D
2,2023-09-22,R40,PADD 4,EPM0F,Finished Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0F_YPB_R40_MBBLD,Rocky Mountain (PADD 4) Blender Net Production...,201,MBBL/D
3,2023-09-22,NUS,U.S.,EPPK,Kerosene,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPPK_YPB_NUS_MBBLD,U.S. Blender Net Production of Kerosene (Thous...,-1,MBBL/D
4,2023-09-22,NUS,U.S.,EPM0RO,Finished Motor Gasoline Reformulated Other,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0RO_YPB_NUS_MBBLD,U.S. Blender Net Production of Other Reformula...,0,MBBL/D


In [32]:
# Combined all blender dataframes vertically
blender_combined = pd.concat([blending_df1, blending_df2, blending_df3], axis=0)
blender_combined.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2014-11-28,NUS,U.S.,EPD0,Distillate Fuel Oil,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPD0_YPB_NUS_MBBLD,U.S. Blender Net Production of Distillate Fuel...,21,MBBL/D
1,2014-11-28,R20,PADD 2,EPM0F,Finished Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0F_YPB_R20_MBBLD,Midwest (PADD 2) Blender Net Production of Fin...,2166,MBBL/D
2,2014-11-28,NUS,U.S.,EPM0CO,Other Conventional Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0CO_YPB_NUS_MBBLD,U.S. Blender Net Production of Other Conventio...,-348,MBBL/D
3,2014-11-28,NUS,U.S.,EPD00H,"Distillate Fuel Oil, Greater Than 500 ppm Sulfur",YPB,Motor Gasoline Terminal Blenders Net Production,W_EPD00H_YPB_NUS_MBBLD,U.S. Blender Net Production of Distillate Fuel...,-21,MBBL/D
4,2014-11-28,NUS,U.S.,EPM0R,Reformulated Motor Gasoline,YPB,Motor Gasoline Terminal Blenders Net Production,W_EPM0R_YPB_NUS_MBBLD,U.S. Blender Net Production of Reformulated Mo...,2989,MBBL/D


In [33]:
# Group by week and sum the total weekly blending volume
blender_total = blender_combined.groupby(by='period')['value'].sum()
blender_total

period
2010-06-04    26749
2010-06-11    27442
2010-06-18    27717
2010-06-25    27856
2010-07-02    28932
              ...  
2023-08-25    39577
2023-09-01    38779
2023-09-08    38370
2023-09-15    38315
2023-09-22    38475
Name: value, Length: 695, dtype: int64

In [34]:
# Reset the index and rename the column name 
blender_weekly_df = blender_total.reset_index().rename(columns={'value':'weekly_total'})
blender_weekly_df.tail()

,period,weekly_total
690,2023-08-25,39577
691,2023-09-01,38779
692,2023-09-08,38370
693,2023-09-15,38315
694,2023-09-22,38475


### 3. Weekly imports and exports
- Note: Only include crude oil and petroleum products

In [35]:
# Set up API query for weekly blender net production (from 2010-06-04 to 2013-11-30)
api_url = f"https://api.eia.gov/v2/petroleum/move/wkly/data/?frequency=weekly&data[0]=value&facets[product][]=EP00&facets[product][]=EPC0&facets[process][]=EEX&facets[process][]=IM0&facets[process][]=IMS&facets[process][]=IMU&facets[process][]=IMX&start=2010-06-04&end=2013-11-30&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
import_export_data1 = r.json()

In [36]:
# Convert the data into DataFrame
import_export_df1 = pd.DataFrame(import_export_data1['response']['data'])
import_export_df1.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2013-11-29,R40-Z00,PADD 4,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP42,Rocky Mountain (PADD 4) Commercial Crude Oil I...,292,MBBL/D
1,2013-11-29,R50-Z00,PADD 5,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R50-Z00_2,West Coast (PADD 5) Imports of Crude Oil and P...,1040,MBBL/D
2,2013-11-29,R40-Z00,PADD 4,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R40-Z00_2,Rocky Mountain (PADD 4) Imports of Crude Oil a...,306,MBBL/D
3,2013-11-29,NUS-Z00,U.S.,EPC0,Crude Oil,EEX,Exports,WCREXUS2,U.S. Exports of Crude Oil (Thousand Barrels pe...,57,MBBL/D
4,2013-11-29,R50-Z00,PADD 5,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP52,West Coast (PADD 5) Commercial Crude Oil Impor...,899,MBBL/D


In [37]:
# Set up API query for weekly blender net production (from 2013-12-01 to 2017-03-31)
api_url = f"https://api.eia.gov/v2/petroleum/move/wkly/data/?frequency=weekly&data[0]=value&facets[product][]=EP00&facets[product][]=EPC0&facets[process][]=EEX&facets[process][]=IM0&facets[process][]=IMS&facets[process][]=IMU&facets[process][]=IMX&start=2013-12-01&end=2017-03-31&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
import_export_data2 = r.json()

In [38]:
# Convert the data into DataFrame
import_export_df2 = pd.DataFrame(import_export_data2['response']['data'])
import_export_df2.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2017-03-31,NUS-Z00,U.S.,EPC0,Crude Oil,IMU,Imports by Others for SPR,W_EPC0_IMU_NUS-Z00_MBBLD,U.S. Crude Oil Imports for SPR by Others (Thou...,0,MBBL/D
1,2017-03-31,NUS-Z00,U.S.,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMUS2,U.S. Commercial Crude Oil Imports Excluding SP...,7850,MBBL/D
2,2017-03-31,NUS-Z00,U.S.,EPC0,Crude Oil,IM0,Imports,WCRIMUS2,U.S. Imports of Crude Oil (Thousand Barrels pe...,7850,MBBL/D
3,2017-03-31,R50-Z00,PADD 5,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R50-Z00_2,West Coast (PADD 5) Imports of Crude Oil and P...,1353,MBBL/D
4,2017-03-31,NUS-Z00,U.S.,EP00,Crude Oil and Petroleum Products,EEX,Exports,WTTEXUS2,U.S. Exports of Crude Oil and Petroleum Produc...,5310,MBBL/D


In [39]:
# Set up API query for weekly blender net production (from 2017-04-01 to 2020-09-30)
api_url = f"https://api.eia.gov/v2/petroleum/move/wkly/data/?frequency=weekly&data[0]=value&facets[product][]=EP00&facets[product][]=EPC0&facets[process][]=EEX&facets[process][]=IM0&facets[process][]=IMS&facets[process][]=IMU&facets[process][]=IMX&start=2017-04-01&end=2020-09-30&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
import_export_data3 = r.json()

In [40]:
# Convert the data into DataFrame
import_export_df3 = pd.DataFrame(import_export_data3['response']['data'])
import_export_df3.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2020-09-25,R40-Z00,PADD 4,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R40-Z00_2,Rocky Mountain (PADD 4) Imports of Crude Oil a...,364,MBBL/D
1,2020-09-25,R20-Z00,PADD 2,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP22,Midwest (PADD 2) Commercial Crude Oil Imports ...,2394,MBBL/D
2,2020-09-25,R40-Z00,PADD 4,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP42,Rocky Mountain (PADD 4) Commercial Crude Oil I...,345,MBBL/D
3,2020-09-25,NUS-Z00,U.S.,EPC0,Crude Oil,EEX,Exports,WCREXUS2,U.S. Exports of Crude Oil (Thousand Barrels pe...,3512,MBBL/D
4,2020-09-25,R50-Z00,PADD 5,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP52,West Coast (PADD 5) Commercial Crude Oil Impor...,965,MBBL/D


In [41]:
# Set up API query for weekly blender net production (from 2020-10-01 to 2023-09-22)
api_url = f"https://api.eia.gov/v2/petroleum/move/wkly/data/?frequency=weekly&data[0]=value&facets[product][]=EP00&facets[product][]=EPC0&facets[process][]=EEX&facets[process][]=IM0&facets[process][]=IMS&facets[process][]=IMU&facets[process][]=IMX&start=2020-10-01&end=2023-09-22&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
import_export_data4 = r.json()

In [42]:
# Convert the data into DataFrame
import_export_df4 = pd.DataFrame(import_export_data4['response']['data'])
import_export_df4.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2023-09-22,R40-Z00,PADD 4,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP42,Rocky Mountain (PADD 4) Commercial Crude Oil I...,529,MBBL/D
1,2023-09-22,R10-Z00,PADD 1,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP12,East Coast (PADD 1) Commercial Crude Oil Impor...,571,MBBL/D
2,2023-09-22,R20-Z00,PADD 2,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP22,Midwest (PADD 2) Commercial Crude Oil Imports ...,3079,MBBL/D
3,2023-09-22,R20-Z00,PADD 2,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R20-Z00_2,Midwest (PADD 2) Imports of Crude Oil and Petr...,3165,MBBL/D
4,2023-09-22,NUS-Z00,U.S.,EPC0,Crude Oil,IM0,Imports,WCRIMUS2,U.S. Imports of Crude Oil (Thousand Barrels pe...,7229,MBBL/D


In [43]:
# Combined all blender dataframes vertically
importExport_combined = pd.concat([import_export_df1,import_export_df2, import_export_df3, import_export_df4], axis=0)
importExport_combined.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2013-11-29,R40-Z00,PADD 4,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP42,Rocky Mountain (PADD 4) Commercial Crude Oil I...,292,MBBL/D
1,2013-11-29,R50-Z00,PADD 5,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R50-Z00_2,West Coast (PADD 5) Imports of Crude Oil and P...,1040,MBBL/D
2,2013-11-29,R40-Z00,PADD 4,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R40-Z00_2,Rocky Mountain (PADD 4) Imports of Crude Oil a...,306,MBBL/D
3,2013-11-29,NUS-Z00,U.S.,EPC0,Crude Oil,EEX,Exports,WCREXUS2,U.S. Exports of Crude Oil (Thousand Barrels pe...,57,MBBL/D
4,2013-11-29,R50-Z00,PADD 5,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP52,West Coast (PADD 5) Commercial Crude Oil Impor...,899,MBBL/D


In [44]:
# Print out unique values
importExport_combined["process-name"].unique()

array(['Imports Excluding SPR', 'Imports', 'Exports', 'Imports by SPR',
       'Imports by Others for SPR'], dtype=object)

In [45]:
# filter out weekly export data
export_df = importExport_combined.loc[importExport_combined["process-name"] == 'Exports']
export_df.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
3,2013-11-29,NUS-Z00,U.S.,EPC0,Crude Oil,EEX,Exports,WCREXUS2,U.S. Exports of Crude Oil (Thousand Barrels pe...,57,MBBL/D
13,2013-11-29,NUS-Z00,U.S.,EP00,Crude Oil and Petroleum Products,EEX,Exports,WTTEXUS2,U.S. Exports of Crude Oil and Petroleum Produc...,3443,MBBL/D
28,2013-11-22,NUS-Z00,U.S.,EPC0,Crude Oil,EEX,Exports,WCREXUS2,U.S. Exports of Crude Oil (Thousand Barrels pe...,57,MBBL/D
29,2013-11-22,NUS-Z00,U.S.,EP00,Crude Oil and Petroleum Products,EEX,Exports,WTTEXUS2,U.S. Exports of Crude Oil and Petroleum Produc...,3443,MBBL/D
42,2013-11-15,NUS-Z00,U.S.,EP00,Crude Oil and Petroleum Products,EEX,Exports,WTTEXUS2,U.S. Exports of Crude Oil and Petroleum Produc...,3379,MBBL/D


In [46]:
# Group by week and sum the total weekly export volume
export_total = export_df.groupby(by='period')['value'].sum()
export_total

period
2010-06-04     1819
2010-06-11     1819
2010-06-18     2058
2010-06-25     2058
2010-07-02     2058
              ...  
2023-08-25    14937
2023-09-01    16358
2023-09-08    12142
2023-09-15    15761
2023-09-22    14347
Name: value, Length: 695, dtype: int64

In [47]:
# Reset the index and rename the column name 
export_weekly_df = export_total.reset_index().rename(columns={'value':'weekly_total'})
export_weekly_df.tail()

,period,weekly_total
690,2023-08-25,14937
691,2023-09-01,16358
692,2023-09-08,12142
693,2023-09-15,15761
694,2023-09-22,14347


In [48]:
# filter out weekly import data
import_df = importExport_combined.loc[importExport_combined["process-name"] != 'Exports']
import_df.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2013-11-29,R40-Z00,PADD 4,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP42,Rocky Mountain (PADD 4) Commercial Crude Oil I...,292,MBBL/D
1,2013-11-29,R50-Z00,PADD 5,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R50-Z00_2,West Coast (PADD 5) Imports of Crude Oil and P...,1040,MBBL/D
2,2013-11-29,R40-Z00,PADD 4,EP00,Crude Oil and Petroleum Products,IM0,Imports,WTTIM_R40-Z00_2,Rocky Mountain (PADD 4) Imports of Crude Oil a...,306,MBBL/D
4,2013-11-29,R50-Z00,PADD 5,EPC0,Crude Oil,IMX,Imports Excluding SPR,WCEIMP52,West Coast (PADD 5) Commercial Crude Oil Impor...,899,MBBL/D
5,2013-11-29,NUS-Z00,U.S.,EPC0,Crude Oil,IMS,Imports by SPR,WCSIMUS2,U.S. Crude Oil Imports by SPR (Thousand Barrel...,0,MBBL/D


In [49]:
# Group by week and sum the total weekly export volume
import_total = import_df.groupby(by='period')['value'].sum()
import_total

period
2010-06-04    51704
2010-06-11    52709
2010-06-18    55208
2010-06-25    52633
2010-07-02    53085
              ...  
2023-08-25    37300
2023-09-01    37976
2023-09-08    41712
2023-09-15    36358
2023-09-22    38945
Name: value, Length: 695, dtype: int64

In [50]:
# Reset the index and rename the column name 
import_weekly_df = import_total.reset_index().rename(columns={'value':'weekly_total'})
import_weekly_df.tail()

,period,weekly_total
690,2023-08-25,37300
691,2023-09-01,37976
692,2023-09-08,41712
693,2023-09-15,36358
694,2023-09-22,38945


### 4. Weekly Stock
- Note: The following crude oil stock data is only include its primary delivery point Cushing, Oklahoma for WTI crude oil futures and main US storage crude oil storage place,Gulf Coast (from PADD3)

In [51]:
# Set up API query for Crude Oil Production in USA
api_url = f"https://api.eia.gov/v2/petroleum/stoc/wstk/data/?frequency=weekly&data[0]=value&facets[product][]=EP00&facets[product][]=EPC0&facets[series][]=WCESTP31&facets[series][]=W_EPC0_SAX_R30_MBBL&facets[series][]=W_EPC0_SAX_YCUOK_MBBL&start=2010-06-04&end=2023-09-22&sort[0][column]=period&sort[0][direction]=desc&offset=0&length=5000&api_key={api_key_eia}"
r = requests.get(api_url)
stock_data = r.json()

In [52]:
# Convert the data into DataFrame
stock_df = pd.DataFrame(stock_data['response']['data'])
stock_df.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2023-09-22,YCUOK,NA,EPC0,Crude Oil,SAX,Ending Stocks Excluding SPR,W_EPC0_SAX_YCUOK_MBBL,"Cushing, OK Ending Stocks excluding SPR of Cru...",21958,MBBL
1,2023-09-22,R30,PADD 3,EPC0,Crude Oil,SAX,Ending Stocks Excluding SPR,WCESTP31,Gulf Coast (PADD 3) Ending Stocks excluding SP...,237997,MBBL
2,2023-09-15,R30,PADD 3,EPC0,Crude Oil,SAX,Ending Stocks Excluding SPR,WCESTP31,Gulf Coast (PADD 3) Ending Stocks excluding SP...,239077,MBBL
3,2023-09-15,YCUOK,NA,EPC0,Crude Oil,SAX,Ending Stocks Excluding SPR,W_EPC0_SAX_YCUOK_MBBL,"Cushing, OK Ending Stocks excluding SPR of Cru...",22901,MBBL
4,2023-09-08,YCUOK,NA,EPC0,Crude Oil,SAX,Ending Stocks Excluding SPR,W_EPC0_SAX_YCUOK_MBBL,"Cushing, OK Ending Stocks excluding SPR of Cru...",24965,MBBL


In [53]:
# Group by "period" column and calcuate the total amount of crude oil stock
stock_total = stock_df.groupby(stock_df['period'])['value'].sum()
stock_total

period
2010-06-04    377915
2010-06-11    378613
2010-06-18    381991
2010-06-25    386192
2010-07-02    384178
               ...  
2023-08-25    263467
2023-09-01    258838
2023-09-08    263664
2023-09-15    261978
2023-09-22    259955
Name: value, Length: 695, dtype: int64

In [54]:
# Reset the index and rename the column name 
stock_weekly_df = stock_total.reset_index().rename(columns={'value':'weekly_total'})
stock_weekly_df.head()

,period,weekly_total
0,2010-06-04,377915
1,2010-06-11,378613
2,2010-06-18,381991
3,2010-06-25,386192
4,2010-07-02,384178


## Read all other features from CSV files

In [58]:
# Read the original CSV file
natural_gas_df = pd.read_csv("Resources/Henry_Hub_Natural_Gas_Spot_Price_weekly.csv") # Henry Hub Natural Gas Spot Price(weekly)
brent_df = pd.read_csv("Resources/brent_weekly.csv") # crude oil brent spot price (weekly)

In [59]:
# Review natural gas dataframe
natural_gas_df.head()

,Week of,Henry Hub Natural Gas Spot Price Dollars per Million Btu
0,9/22/23,2.62
1,9/15/23,2.71
2,9/8/23,2.52
3,9/1/23,2.57
4,8/25/23,2.53


In [60]:
# Filter the data by date range (from 2010 Jun - 2023 Sep)
f_natural_gas_df = natural_gas_df.iloc[:695]

# Sort the dataframe by column "date" in ascending order
f_natural_gas_df = f_natural_gas_df.sort_index(ascending=False).reset_index(drop=True)
f_natural_gas_df.tail()

,Week of,Henry Hub Natural Gas Spot Price Dollars per Million Btu
690,8/25/23,2.53
691,9/1/23,2.57
692,9/8/23,2.52
693,9/15/23,2.71
694,9/22/23,2.62


In [61]:
# Review crude oil Brent dataframe
brent_df.head()

,date,value
0,2023-09-22,94.86
1,2023-09-15,93.87
2,2023-09-08,91.29
3,2023-09-01,87.57
4,2023-08-25,84.71


In [62]:
# Filter the data by date range (from 2020 Jan - 2023 May)
f_brent_df = brent_df.iloc[:695]

# Sort the dataframe by column "date" in ascending order
f_brent_df = f_brent_df.sort_values(by='date', ascending=True).reset_index(drop=True)
f_brent_df.tail()

,date,value
690,2023-08-25,84.71
691,2023-09-01,87.57
692,2023-09-08,91.29
693,2023-09-15,93.87
694,2023-09-22,94.86


## Combine all features and target variable into one DataFrame

In [63]:
# Copy all features to new dataframe
oil_model_weekly_df['sum_refiner'] = refiner_weekly_df['weekly_total']
oil_model_weekly_df['sum_blender'] = blender_weekly_df['weekly_total']
oil_model_weekly_df['sum_export'] = export_weekly_df['weekly_total']
oil_model_weekly_df['sum_import'] = import_weekly_df['weekly_total']
oil_model_weekly_df['sum_stock'] = stock_weekly_df['weekly_total']

oil_model_weekly_df['natural_gas_spot'] = f_natural_gas_df['Henry Hub Natural Gas Spot Price Dollars per Million Btu']
oil_model_weekly_df['brent_spot'] = f_brent_df['value']

#Review Dataframe
oil_model_weekly_df.head()

,date,oil_spot_price_weekly,sum_refiner,sum_blender,sum_export,sum_import,sum_stock,natural_gas_spot,brent_spot
0,2010-06-06,72.9075,24800,26749,1819,51704,377915,4.44,72.71
1,2010-06-13,73.4360,24507,27442,1819,52709,378613,4.73,72.76
2,2010-06-20,76.7000,24736,27717,2058,55208,381991,5.10,76.22
3,2010-06-27,77.0600,25214,27856,2058,52633,386192,4.93,76.64
4,2010-07-04,74.9580,24559,28932,2058,53085,384178,4.66,73.86


In [64]:
# Write the DataFrame to a CSV file
oil_model_weekly_df.to_csv("Outputs/oil_model_weekly_data.csv", index=False)